In [ ]:
import os
from google.cloud import bigquery
import pandas as pd
import matplotlib.pyplot as plt

PROJECT_ID = "nyc-taxi-pipeline-2026"
os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = '/Users/mohammedpathariya/Docs/IUB Docs/Projects/nyc_taxi_rides/terraform/google_credentials.json'

client = bigquery.Client()
print(f"Connected to BigQuery project: {PROJECT_ID}")

## Raw Data Preview

In [ ]:
# Previewing the 'Raw' Bronze Layer for Jan 2024
yellow_preview_query = """
SELECT *
FROM `nyc-taxi-pipeline-2026.trips_data_all.yellow_tripdata_external`
WHERE tpep_pickup_datetime >= '2024-01-01' AND tpep_pickup_datetime < '2024-02-01'
LIMIT 5
"""

green_preview_query = """
SELECT *
FROM `nyc-taxi-pipeline-2026.trips_data_all.green_tripdata_external`
WHERE lpep_pickup_datetime >= '2024-01-01' AND lpep_pickup_datetime < '2024-02-01'
LIMIT 5
"""

print("--- Yellow Taxi Raw Preview (Jan 2024) ---")
display(client.query(yellow_preview_query).to_dataframe())

print("\n--- Green Taxi Raw Preview (Jan 2024) ---")
display(client.query(green_preview_query).to_dataframe())

### Analysis of Raw Data Previews (Jan 2024)

#### 1. Column Prefix Divergence
* **Yellow Taxi:** Uses the `tpep_` prefix (e.g., `tpep_pickup_datetime`).
* **Green Taxi:** Uses the `lpep_` prefix (e.g., `lpep_pickup_datetime`).
* **Engineering Impact:** Our staging models must alias these to a unified name (like `pickup_datetime`) to allow for a vertical `UNION` in the final fact table.

#### 2. Unique Field Presence
* **Green Taxi Only:** Contains `ehail_fee` (largely NaN/NULL in the sample) and `trip_type`.
* **Yellow Taxi Only:** Contains `Airport_fee`.
* **Engineering Impact:** We need to decide whether to drop these unique fields or include them as NULLs in the "opposite" fleet's model to maintain a consistent schema.

#### 3. Spatial Granularity
* Both fleets use `PULocationID` and `DOLocationID` (numeric IDs).
* **Observation:** The sample shows IDs like `249`, `166`, `68`, and `174`. Without the `taxi_zone_lookup` seed, these rows are spatially anonymous.

#### 4. Flag Variations
* Both datasets include `store_and_fwd_flag` (e.g., 'N'), indicating whether the trip data was held in the vehicle's memory before transmission.

## Data EDA

In [ ]:
yellow_vendor_query = """
SELECT 
    vendorid,
    COUNT(*) AS total_trips,
    COUNTIF(passenger_count IS NULL) AS null_passenger_count,
    ROUND(COUNTIF(passenger_count IS NULL) / COUNT(*) * 100, 2) AS null_percentage
FROM `nyc-taxi-pipeline-2026.trips_data_all.yellow_tripdata_external`
GROUP BY 1
ORDER BY total_trips DESC
"""

df_yellow_vendor = client.query(yellow_vendor_query).to_dataframe()
print("Yellow Taxi Vendor Analysis:")
df_yellow_vendor

In [ ]:
green_vendor_query = """
SELECT 
    vendorid,
    COUNT(*) AS total_trips,
    COUNTIF(passenger_count IS NULL) AS null_passenger_count,
    ROUND(COUNTIF(passenger_count IS NULL) / COUNT(*) * 100, 2) AS null_percentage
FROM `nyc-taxi-pipeline-2026.trips_data_all.green_tripdata_external`
GROUP BY 1
ORDER BY total_trips DESC
"""

df_green_vendor = client.query(green_vendor_query).to_dataframe()
print("Green Taxi Vendor Analysis:")
df_green_vendor

In [ ]:
# Combining Yellow and Green for a high-level range check
outlier_query = """
SELECT 
    'Yellow' as taxi_type,
    MIN(trip_distance) as min_dist, MAX(trip_distance) as max_dist, AVG(trip_distance) as avg_dist,
    MIN(total_amount) as min_fare, MAX(total_amount) as max_fare
FROM `nyc-taxi-pipeline-2026.trips_data_all.yellow_tripdata_external`
WHERE tpep_pickup_datetime BETWEEN '2024-01-01' AND '2025-12-31'
UNION ALL
SELECT 
    'Green' as taxi_type,
    MIN(trip_distance) as min_dist, MAX(trip_distance) as max_dist, AVG(trip_distance) as avg_dist,
    MIN(total_amount) as min_fare, MAX(total_amount) as max_fare
FROM `nyc-taxi-pipeline-2026.trips_data_all.green_tripdata_external`
WHERE lpep_pickup_datetime BETWEEN '2024-01-01' AND '2025-12-31'
"""

df_outliers = client.query(outlier_query).to_dataframe()
df_outliers

In [ ]:
green_analysis_query = """
SELECT 
    vendorid,
    COUNT(*) AS total_trips,
    COUNTIF(passenger_count IS NULL) AS null_passengers,
    ROUND(COUNTIF(passenger_count IS NULL) / COUNT(*) * 100, 2) AS null_pct,
    COUNTIF(trip_type = 2) AS dispatch_trips,
    ROUND(COUNTIF(trip_type = 2) / COUNT(*) * 100, 2) AS dispatch_pct
FROM `nyc-taxi-pipeline-2026.trips_data_all.green_tripdata_external`
GROUP BY 1
ORDER BY total_trips DESC
"""

df_green_analysis = client.query(green_analysis_query).to_dataframe()
print("Green Taxi Detailed Analysis:")
df_green_analysis

In [ ]:
# Visualization of Yellow Taxi fare distribution
viz_query = """
SELECT total_amount 
FROM `nyc-taxi-pipeline-2026.trips_data_all.yellow_tripdata_external`
WHERE tpep_pickup_datetime BETWEEN '2024-01-01' AND '2024-01-31'
AND total_amount > 0 AND total_amount < 100
LIMIT 100000
"""
df_viz = client.query(viz_query).to_dataframe()

plt.figure(figsize=(10, 6))
plt.hist(df_viz['total_amount'], bins=50, color='gold', edgecolor='black')
plt.title('Yellow Taxi Fare Distribution (Jan 2024 - Subsampled)')
plt.xlabel('Fare Amount ($)')
plt.ylabel('Frequency')
plt.grid(axis='y', alpha=0.3)
plt.show()

## Summary of Bronze Layer Exploration & Data Profiling

### 1. The Provider Problem (Vendor Analysis)
* **Cross-Fleet Consistency:** Vendor 2 (VeriFone) is the dominant provider but carries the highest data gaps across both fleets, with an 18.3% NULL rate in Yellow and a 4.14% NULL rate in Green.
* **Green Fleet Specialization:** Approximately 5.1% of Green Taxi trips are identified as 'Dispatch' (Type 2), while the remainder are street hails.
* **Ghost Vendors:** Vendor 6 remains a high-risk data source with a 100% NULL rate for passenger counts across both datasets.

### 2. Project Scope & Temporal Constraints
To ensure analytical integrity and remove hardware-related "drift," the following temporal constraints are established:
* **In-Scope:** January 1, 2024, to December 31, 2025.
* **Excluded Noise:** All legacy records from 2002, 2007, 2008, and 2009, as well as premature 2026 logs.

### 3. Outlier "Sanity Bounds" for Silver Layer
Based on statistical profiling, the following filtering thresholds will be applied in the dbt staging models to remove sensor noise and data entry errors:
* **Distance:** `trip_distance` must be $> 0$ and $< 100$ miles (to remove the 398k-mile outliers).
* **Fares:** `total_amount` must be $> 0$ and $< $1,000 (to remove negative voids and $863k glitches).

### 4. Transformation Logic (Silver Layer Roadmap)
* **Standardization:** Map disparate vendor IDs and consolidate pickup/dropoff timestamp columns into a unified schema.
* **Imputation:** Use `COALESCE(passenger_count, 1)` to handle missing values, as 1 is the statistical mode for NYC taxi trips.
* **Enrichment:** Join with the `taxi_zone_lookup` seed table to transition from numeric IDs to borough and zone names.